# Analysis of Treasury Bill Pricing and Interest Rate Risk

## Background
Treasury bills, otherwise known as T-Bills, are financial instruments with short-term due dates ranging from a few days to 52 weeks. They are fixed-income investments with zero coupons, meaning no coupon payments are made during the term. Instead, the price of treasury bills is calculated so that the bill receiver gets the face (par) value ($V_{P}$) at the end of the term. 

<div>
    <center>
        <img src="figs/Fig-ZC-Schematic.png" width="440"/>
    </center>
</div>

### Pricing
A zero-coupon T-bill with an annual effective market interest rate $\bar{r}$, which is specified at the time of purchase, and a term of T years has a _fair price_ of:

$$
V_{B} = \mathcal{D}_{T,0}^{-1}\cdot{V_{P}}
$$

where $\mathcal{D}_{T}$ denotes the discount factor from the time of the auction to the term of T-bill, i.e., $0\rightarrow{T}$. Typically, a constant averaged rate of interest is used to compute the discount factor:

$$
\mathcal{D}_{T,0} = (1+\bar{r})^{T}
$$

However, dynamic formulations of the discount factor with period-specific interest rates can also be used:

$$
\mathcal{D}_{T,0} = \left[\prod_{j=0}^{T-1}\left(1+r_{j+1,j}\right)\right]
$$

where $r_{j+1,j}$ terms are called short rates, i.e., the interest rate earned between period $j\rightarrow{j+1}$. Treasury bills are released for specific periods $T=$ 4, 8, 13, 26, and 52 weeks. The fair price $V_{B}$ of the T-bill is the future face value $V_{P}$ that is discounted to today’s value by the effective market interest rate $\bar{r}$, which is assumed constant over the lifetime of the treasury bill.

## Project
You purchased a 52-week T-bill with a par value $V_{P}=100.0$ USD and an effective market rate of interest $\bar{r}$ (annualized). However, four weeks (28-days) after purchasing the T-bill, the interest rate is now $\bar{r}^{\prime}$. Should you `sell` the T-bill before it matures or `hold` the T-bill to term?

### Tasks
 1. Load, and analyze historical T-bill interest rate data sets from 2019 to 2023 computed daily by [Federal Reserve Bank of New York](https://www.newyorkfed.org). Store these data in a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) where the year, e.g., `"2022"` are the keys and the data sets of the values. 
 1. Develop a strategy to decide between a `hold` or `sell` action for your T-bill.
 1. Backtest your stratetgy using historical interest rate data for 52-week T-bills
     * Implement your strategy and compute the number of sucess and failures of strategy

## Setup
In the following code block we setup the computational aspects of the problem by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this excercise and custom types to model the components of our problem.

### Packages
`Include.jl` loads several external packages that we will use for our excercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general purpose data science tool.
* [Plots.jl](https://docs.juliaplots.org/stable/) is a plotting library in [Julia](https://julialang.org).

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/) into the notebook:

##### `loadratesfile(year::String = "2022") -> DataFrame`

> This function takes a [String](https://docs.julialang.org/en/v1/manual/strings/) encoding the year in the `YYYY` format, e.g., `"2019"` or `"2020"` and returns a [DataFrame](https://dataframes.juliadata.org/stable/) holding the daily interest rates for US Treasury Bills for various durations found on [Treasury.gov](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_bill_rates). This project has data from `"2019"`, `"2020"`, `"2021"`, `"2022"` and `"2022"`. The year parameter has a default value of `"2022"`.

##### `build(model::Type{MyUSTreasuryBillModel}, data::NamedTuple) -> MyUSTreasuryBillModel`

> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument (the par value, the rate and the duration of the T-bill) and returns an instance of the `MyUSTreasuryBillModel` custom type.

##### `build(model::Type{MySymmetricBinaryLatticeModel}, data::NamedTuple)::MySymmetricBinaryLatticeModel` and `build(model::Type{MyBinaryLatticeNodeModel}, data::NamedTuple)::MyBinaryLatticeNodeModel` 

> These factory functions take information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument and construct a node or the binomial short-rate lattice model, respectively. 

##### `price(model::MyUSTreasuryBillModel, compounding::T) -> MyUSTreasuryBillModel where T <: AbstractCompoundingModel`

> This function takes a `MyUSTreasuryBillModel` instance, and a compounding model and returns an updated `MyUSTreasuryBillModel` instance that holds the price $V_{B}$ value in the `price` field of the `MyUSTreasuryBillModel` instance.

##### `𝔼(model::MySymmetricBinaryLatticeModel; level::Int = 0) -> Float64` and `𝕍(model::MySymmetricBinaryLatticeModel; level::Int = 0) -> Float64`

> These functions take a`MySymmetricBinaryLatticeModel` instance, and the level and the tree (`0`-based) and compute the [expectation](https://en.wikipedia.org/wiki/Expected_value) and [variance](https://en.wikipedia.org/wiki/Variance) of the short-rates for the given level (time-step) of the lattice. 

### Types
`Include.jl` loads some [problem-specific types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of T-bill pricing. 

* `MyUSTreasuryBillModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$ and the interest rate $\bar{r}$ for a zero-coupon treasury bill. You construct a `MyUSTreasuryBillModel` instance using the `build` method described above.
* `DiscreteCompoundingModel` and `ContinuousCompoundingModel` are [immutable types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that let our code know which compounding model we wish to use.
* The `MyBinaryLatticeNodeModel` and `MySymmetricBinaryLatticeModel` mutable types hold information about the nodes, and the binomial lattice, respectively, for the short-rate calculations. 

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-1`


## Set constant values 
We will need several constants, regardless of the year we choose to analyze. First, we will always analyze 52-week data, so we encode this choice in the `key` variable. Next, we'll assume a discrete compounding model; thus, we instantiate an instance of the `DiscreteCompoundingModel` type and then store this instance in the `compounding` variable. Lastly, the T-bill’s face (par) value will always be $V_{P}$ = 100.0 USD.

In [9]:
key = Symbol("52 WEEKS BANK DISCOUNT");
compounding = DiscreteCompoundingModel();
Vₚ = 100.0;

## Task 1: Load and analyze historical interest rate ranges
The rates are based on recently auctioned bills with various maturities and collected by the Federal Reserve Bank of New York every business day. The Bank Discount rate is based on par value and discount amount, while the Coupon Equivalent is based on purchase price and discount, and can be used to compare yields.

### Load the historical rate data into a dictionary:

In [5]:
rate_data_dictionary = Dict{String,DataFrame}();
years = ["2019","2020","2021","2022","2023"];
for year ∈ years
    rate_data_dictionary[year] = loadratesfile(year=year);
end

### Select a year of interest rate data to analyze:
Let’s choose a year from `2019` to `2023` and save our selection in the variable `selected_year`. The year value should have the `YYYY` format, for example, `2022`. We then extract the corresponding rate values for the `selected_year` from the `rate_data_dictionary`. The rate data will be stored in the `dataset` variable, which is a `DataFrame`.

In [3]:
selected_year = "2022"
dataset = rate_data_dictionary[selected_year];

### What is the range of rates for the year we have selected?

In [12]:
minimum_value = dataset[:,key] |> minimum;
maximim_value = dataset[:,key] |> maximum;

## Task 2: Develop a Hold versus Sell Strategy for Treasury Bills
Whether to `hold` or `sell` a security depends on your investment goals. Treasury securities, like T-bills, are generally considered safe, short-term investments. When you buy these securities, you receive their face value at the end of the term. However, you won’t have access to the invested funds until that time. If you’re concerned about needing access to the funds before the term ends, such as for unexpected expenses, you could sell the T-bill before maturity and potentially earn a capital gain or loss.

Suppose you have two possible actions $a\in\left\{\text{sell},\text{hold}\right\}$ and you implement the following trade policy:
* You implement the `sell` action if the treasury bill experiences an $\alpha$% gain or a $\beta$ loss
* Otherwise, you implement the `hold` action.